In [139]:
#distance square
def dist_sq(a, b, axis = 1):
    return np.sum((a-b)**2,axis)
#distance square for every point to the centroid
def point_sq(data,index):
    Dist=np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        Dist[i]=(dist_sq(data.ix[index],data.ix[i]))
    return Dist
def dist_prob(Dist):
    return (Dist/np.sum(Dist))

In [164]:
def kmeansplusplus(data, k, d):
    centroid=np.zeros([k,d])
    index=np.random.choice(data.shape[0],1)
    centroid[0]=data.ix[index]
    for number in range(k-1):
        distance=point_sq(data,index)
        prob=dist_prob(distance).tolist()
        prob[index]=0
        index=np.random.choice(data.shape[0],1,p=prob)
        centroid[number+1]=data.ix[index]
    return centroid
    

In [189]:
from numpy.core.umath_tests import inner1d
def kmeansplusplus(data, k, d):
    centroid=np.zeros([k,d])
    index=np.random.choice(data.shape[0],1)
    centroid[0]=data.ix[index]
    for number in range(k-1):
        distance=point_sq(data,index)
        prob=dist_prob(distance).tolist()
        #prob[index]=0
        index=np.random.choice(data.shape[0],1,p=prob)
        centroid[number+1]=data.ix[index]
    return centroid

In [190]:
#test 
data=DataFrame([np.random.random(100),np.random.random(100)]).transpose()
kmeansplusplus(data, 10, 2)

array([[ 0.07421358,  0.50333841],
       [ 0.8751341 ,  0.55550574],
       [ 0.07421358,  0.50333841],
       [ 0.98623634,  0.96817053],
       [ 0.4411158 ,  0.69252865],
       [ 0.05952987,  0.18790654],
       [ 0.74545635,  0.63921622],
       [ 0.09056201,  0.13434502],
       [ 0.98623634,  0.96817053],
       [ 0.13415468,  0.76814779]])